In [1]:
# Cell 1: Initialize and check basic info
from qdrant_client import QdrantClient
from pathlib import Path

# Initialize client (use correct path relative to notebook location)
storage_path = "../vector_db/qdrant_storage"  # Go up one level from notebooks/
client = QdrantClient(path=storage_path)

# Check collections
collections = client.get_collections()
print(f"📚 Collections: {[c.name for c in collections.collections]}")

# Cell 2: Get collection details
collection_name = "resume_data"

if client.collection_exists(collection_name):
    # Get collection info
    collection_info = client.get_collection(collection_name)
    print(f"\n📊 Collection: {collection_name}")
    print(f"   Points count: {collection_info.points_count}")
    print(f"   Vector size: {collection_info.config.params.vectors.size}")
    print(f"   Distance: {collection_info.config.params.vectors.distance}")
else:
    print(f"❌ Collection '{collection_name}' does not exist")

# Cell 3: Query some sample points (if collection exists)
if client.collection_exists(collection_name) and collection_info.points_count > 0:
    # Get first 10 points with scrolling
    records, next_offset = client.scroll(
        collection_name=collection_name,
        limit=10,
        with_payload=True,
        with_vectors=False  # Set True if you want to see vectors
    )
    
    print(f"\n📄 Sample Records (showing {len(records)} of {collection_info.points_count}):")
    for record in records:
        print(f"\n   ID: {record.id}")
        print(f"   Content: {record.payload.get('content', 'N/A')[:100]}...")
        print(f"   Section: {record.payload.get('section_type', 'N/A')}")
        if record.payload.get('metadata'):
            print(f"   Metadata: {record.payload['metadata']}")

# Cell 4: Check for work experience specifically
if client.collection_exists(collection_name):
    from qdrant_client.models import Filter, FieldCondition, MatchValue
    
    # Query work experience entries
    work_records, _ = client.scroll(
        collection_name=collection_name,
        scroll_filter=Filter(
            must=[
                FieldCondition(
                    key="section_type",
                    match=MatchValue(value="work_experience")
                )
            ]
        ),
        limit=5,
        with_payload=True
    )
    
    print(f"\n💼 Work Experience Records (showing {len(work_records)}):")
    for record in work_records:
        metadata = record.payload.get('metadata', {})
        print(f"\n   Position: {metadata.get('position', 'N/A')}")
        print(f"   Company: {metadata.get('company', 'N/A')}")
        print(f"   Dates: {metadata.get('start_date', 'N/A')} - {metadata.get('end_date', 'N/A')}")
        print(f"   Achievement: {record.payload.get('content', 'N/A')[:80]}...")

# Cell 5: Count by section type
if client.collection_exists(collection_name):
    from collections import Counter
    
    # Get all records and count by section
    all_records, _ = client.scroll(
        collection_name=collection_name,
        limit=1000,  # Adjust if you have more
        with_payload=True,
        with_vectors=False
    )
    
    section_counts = Counter(r.payload.get('section_type', 'unknown') for r in all_records)
    
    print(f"\n📈 Document Count by Section Type:")
    for section, count in section_counts.items():
        print(f"   {section}: {count}")
    print(f"\n   TOTAL: {sum(section_counts.values())}")

📚 Collections: ['resume_data']

📊 Collection: resume_data
   Points count: 49
   Vector size: 1536
   Distance: Cosine

📄 Sample Records (showing 10 of 49):

   ID: 00889b43-e7c6-412a-9af5-37d29fc8baa9
   Content: Designed and implemented a reporting tool to pinpoint SKU opportunities at store level, contributing...
   Section: work_experience
   Metadata: {'position': 'Data Analyst', 'company': 'Rubicon Organics', 'start_date': 'March-2023', 'end_date': 'December-2023', 'location': 'Vancouver, BC, Canada', 'industry': 'Data Science/CPG/Cannabis', 'achievement_index': 2}

   ID: 0df677ac-0a47-4566-b117-e4f6125d1d06
   Content: Reduced hard-drive screw defects by 15% (≈$110,000 annual savings) by extracting and cleaning 50,000...
   Section: work_experience
   Metadata: {'position': 'Quality Engineer', 'company': 'IBM', 'start_date': 'December-2017', 'end_date': 'October-2018', 'location': 'Guadalajara, Mexico', 'industry': 'Data Analysis/Technology/Hardware Manufacturing', 'achievement